In [ ]:
import time
import cv2
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"
os.environ["KMP_DUPLICATE_LIB_OK"]="1"

from Code.track import detect
from Code.get_prepared_data_sdd import prepared_data_sdd
from Code.visualize_sdd_annotation import visualizeSddAnnotation
from Code.extract_scene_seg import extract_scene_seg
from Code.preprocess import To_npz
import numpy as np

    
yolo_time_start = time.time()
dataset_resize,changelst , annotation = detect('many_people.mp4')
yolo_time_end = time.time()

traj_data, person_box_data, other_box_data  = prepared_data_sdd(annotation,changelst)

seg_time_start = time.time()
model_path= 'deeplabv3_xception_ade20k_train/frozen_inference_graph.pb'
seg_output= extract_scene_seg(dataset_resize,model_path,every =100)
seg_time_end = time.time()

data=To_npz(8,12,traj_data,seg_output)
np.savez("prepro_fold1/data_test.npz", **data)

simaug_time_start = time.time()
!python Code/test.py prepro_fold1/ packed_models/ best_simaug_model \
--wd 0.001 --runId 0 --obs_len 8 --pred_len 12 --emb_size 32 --enc_hidden_size 256 \
--dec_hidden_size 256 --activation_func tanh --keep_prob 1.0 --num_epochs 30 \
--batch_size 12 --init_lr 0.3 --use_gnn --learning_rate_decay 0.95 --num_epoch_per_decay 5.0 \
--grid_loss_weight 1.0 --grid_reg_loss_weight 0.5 --save_period 3000 \
--scene_h 36 --scene_w 64 --scene_conv_kernel 3 --scene_conv_dim 64 \
--scene_grid_strides 2,4 --use_grids 1,0 --val_grid_num 0 --gpuid 0 --load_best \
--save_output sdd_out.p
simaug_time_end = time.time()


Loading weights from deep_sort_pytorch/deep_sort/deep/checkpoint/ckpt.t7... Done!
YOLOv5  2021-11-21 torch 1.7.1+cu101 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 8192MiB)

YOLOv5  2021-11-21 torch 1.7.1+cu101 CUDA:0 (NVIDIA GeForce RTX 2070 with Max-Q Design, 8192MiB)

Fusing layers... 
Model Summary: 367 layers, 46533693 parameters, 0 gradients


In [2]:
print(f"yolo time : {yolo_time_end-yolo_time_start}")
print(f"seg time : {seg_time_end-seg_time_start}")
print(f"simaug time : {simaug_time_end-simaug_time_start}")
print(f"whole time : {simaug_time_end-yolo_time_start}")

yolo time : 43.69891881942749
seg time : 152.9190957546234
simaug time : 409.57017850875854
whole time : 611.6333334445953


In [ ]:
!echo fsdd_out_0.p,0_0_255 > sdd_run.lst
path= r"C:\Users\melsh\project\GProject\output pictures\resized_videos_frames"
for i,j in dataset_resize.items():
    cv2.imwrite(os.path.join(path, "%s_F_%08d.jpg" % ("stream", i)), j)

In [ ]:
import pickle
with open("seg_output.p", "wb") as f:
      pickle.dump(seg_output, f)
framesData = visualizeSddAnnotation(dataset_resize, changelst, traj_data, person_box_data, other_box_data,1)
with open("framesData.p", "wb") as f:
      pickle.dump(framesData, f)

In [ ]:
!python code/visualize_output.py sdd_run.lst "output pictures/resized_videos_frames" "output pictures/sdd_vis_heatmap/" \
--vis_num 1   --use_heatmap --ordered

In [ ]:
import matplotlib.pyplot as plt
import cv2
import pandas as pd
import os
seg_output = pd.read_pickle(r"seg_output.p")
framesData = pd.read_pickle(r"framesData.p")

In [ ]:
framesData_path= r"C:\Users\melsh\project\GProject\output pictures\framesData"
for i,j in framesData.items():
    plt.imshow(j)
    plt.savefig(os.path.join(framesData_path, "%s.jpg" % ( i)), dpi=100)
    plt.clf()
segmented_path= r"C:\Users\melsh\project\GProject\output pictures\segmented"
for i,j in seg_output.items():
    plt.imshow(j)
    plt.savefig(os.path.join(segmented_path, "%s.jpg" % ( i)), dpi=100)
    plt.clf()

In [ ]:
import time
from Code.track_stream import detect
from Code.queue import get_queue
from Code.get_prepared_data_sdd import prepared_data_sdd
from Code.visualize_sdd_annotation import visualizeSddAnnotation
from Code.extract_scene_seg import extract_scene_seg
from Code.preprocess import To_npz
import cv2
import numpy as np
import os
# cv2.namedWindow("webcam", cv2.WINDOW_NORMAL)
camera = cv2.VideoCapture(0)
for i in range(1):
    os.environ["OMP_NUM_THREADS"] = "1"
    os.environ["OPENBLAS_NUM_THREADS"] = "1"
    os.environ["MKL_NUM_THREADS"] = "1"
    os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
    os.environ["NUMEXPR_NUM_THREADS"] = "1"
    os.environ["KMP_DUPLICATE_LIB_OK"]="1"
    images_try = get_queue(5,12,camera)
    dataset_resize,changelst , annotation = detect(images_try)
    traj_data, person_box_data, other_box_data = prepared_data_sdd(annotation,changelst)
    # framesData = visualizeSddAnnotation(dataset_resize, changelst, traj_data, person_box_data, other_box_data)
    model_path= 'deeplabv3_xception_ade20k_train/frozen_inference_graph.pb'
    seg_output= extract_scene_seg(dataset_resize,model_path,every =2)
    data=To_npz(4,6,traj_data,seg_output)
    np.savez("prepro_fold1/data_test.npz", **data)
    !python Code/test.py prepro_fold1/ packed_models/ best_simaug_model \
    --wd 0.001 --runId 0 --obs_len 4 --pred_len 6 --emb_size 32 --enc_hidden_size 256 \
    --dec_hidden_size 256 --activation_func tanh --keep_prob 1.0 --num_epochs 30 \
    --batch_size 12 --init_lr 0.3 --use_gnn --learning_rate_decay 0.95 --num_epoch_per_decay 5.0 \
    --grid_loss_weight 1.0 --grid_reg_loss_weight 0.5 --save_period 3000 \
    --scene_h 36 --scene_w 64 --scene_conv_kernel 3 --scene_conv_dim 64 \
    --scene_grid_strides 2,4 --use_grids 1,0 --val_grid_num 0 --gpuid 0 --load_best \
    --save_output "sdd_out_{i}.p"

In [ ]:
camera.release()